In [1]:
import datetime as dt
import os
from pathlib import Path

from config import SETTINGS
from pyodm import Node

import seabeepy as sb

In [2]:
# Connect to NodeODM
node = Node.from_url("http://nodeodm")

In [3]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

# Process SeaBee mission data

This notebook performs automatic processing and publishing of data from SeaBee missions. It is designed to run as a "cron job" that will scan and process all flight folders within the specified `base_dirs`.

**Each flight folder must be organised according to the specification [here](https://seabee-no.github.io/documentation/data-upload.html)**.

## 1. User input

In [4]:
# Parent directories containing flight folders to process
base_dirs = [
    r"/home/notebook/shared-seabee-ns9879k/seabirds/2023",
    r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2023",
    r"/home/notebook/shared-seabee-ns9879k/dmc",
]

# Directory for temporary files
temp_dir = r"/home/notebook/cogs/"

In [5]:
# Run info
run_date = dt.datetime.today()
print(f"Processing started: {run_date}")

Processing started: 2023-10-18 11:00:06.299731


## 2. Submit tasks to NodeODM

In [6]:
# Get all potential mission folders for NodeODM
# (i.e. folders containing a 'config.seabee.yaml' and an 'images' subdirectory, but NOT an 'orthophoto' directory)
mission_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "images")
    and not sb.ortho.check_subdir_exists(f.parent, "orthophoto")
]

# Exclude missions already submitted to NodeODM
nodeodm_tasks = sb.ortho.get_nodeodm_tasks(node)
existing_task_paths = [t.info().name for t in nodeodm_tasks]
mission_list = [f for f in mission_list if str(f) not in existing_task_paths]

# Get just those with valid 'config.yaml' files
invalid_config = [f.name for f in mission_list if not sb.ortho.check_config_valid(f)]
invalid_counts = [f.name for f in mission_list if not sb.ortho.check_file_count(f)]
mission_list = [
    f
    for f in mission_list
    if sb.ortho.check_config_valid(f) and sb.ortho.check_file_count(f)
]

# Get just those where 'mosaic' is True in 'config.yaml'
mission_list = [f for f in mission_list if sb.ortho.parse_config(f)["mosaic"]]

print("The following folders have invalid 'config.yaml' files:")
print(invalid_config)

print("\nThe following folders have invalid image counts:")
print(invalid_counts)

print("\nThe following folders are ready to be processed:")
print(mission_list)

The following folders have invalid 'config.yaml' files:
['test_data_set']

The following folders have invalid image counts:
['oslofjorden_lyseren-thermal_202306271117']

The following folders are ready to be processed:
[]


In [7]:
# Process missions
for mission_fold in mission_list:
    mission_name = mission_fold.name
    print(f"\n################\nProcessing: {mission_name}")
    image_fold = os.path.join(mission_fold, "images")
    image_files = sb.ortho.list_images(image_fold, ext="JPG", verbose=False)

    # Add GCPs, if available
    user_gcp_path = os.path.join(mission_fold, "gcp", "gcp_list-ODM.txt")
    req_gcp_path = os.path.join(mission_fold, "gcp", "gcp_list.txt")
    if os.path.isfile(user_gcp_path):
        print("Using GCPs.")
        sb.storage.copy_file(user_gcp_path, req_gcp_path, minio_client, overwrite=False)
        image_files.append(req_gcp_path)

    # Update default options based on 'config.yaml'
    nodeodm_options = sb.ortho.get_nodeodm_options(mission_fold)

    # Send task to NodeODM. Options are documented here: https://docs.opendronemap.org/arguments/
    # Use the mission folder as the name for each task, so lookup is easier for publishing
    task = node.create_task(image_files, nodeodm_options, name=str(mission_fold))

    # The renamed GCP file is not needed after it has been copied to NodeODM
    if os.path.isfile(req_gcp_path):
        sb.storage.delete_file(req_gcp_path, minio_client)

## 3. Transfer NodeODM results back to flight folders on MinIO

In [8]:
# Get completed tasks on NodeODM that have not yet been transferred
nodeodm_tasks = sb.ortho.get_nodeodm_tasks(node)
finished_tasks = [
    t for t in nodeodm_tasks if str(t.info().status) == "TaskStatus.COMPLETED"
]
mission_folders = [
    str(f.parent)
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "images")
    and not sb.ortho.check_subdir_exists(f.parent, "orthophoto")
]
upload_tasks = [t for t in finished_tasks if t.info().name in mission_folders]

print("The following tasks will be copied from NodeODM to the flight folders on MinIO:")
print([t.info().name for t in upload_tasks])

The following tasks will be copied from NodeODM to the flight folders on MinIO:
[]


In [9]:
# Copy to MinIO and remove from NodeODM
for task in upload_tasks:
    task_id = task.info().uuid
    mission_fold = task.info().name
    mission = os.path.basename(mission_fold)
    print("Copying", mission)
    is_copied = sb.storage.copy_nodeodm_results(task_id, mission_fold, minio_client)
    if is_copied:
        print(f"Removing task {task_id}")
        task.remove()
    else:
        print(f"Results was not copied, keeping {task_id}")

## 4. Publish to GeoNode

In [10]:
# Identify datasets for publishing. Folders must contain either an ODM or Pix4D
# original orthophoto (not both) and must not contain a COG named f'{layer_name}.tif'.
# Folders must also have 'config.seabee.yaml' files where 'publish' is True
publish_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.is_publish_ready(f.parent)
    and sb.ortho.parse_config(f.parent)["publish"]
]

print("The following missions will be published to GeoNode:")
print(publish_list)

KeyError: 'grouping'

In [11]:
# Publish
for mission_fold in publish_list:
    mission_name = mission_fold.name
    print(f"\n################\nProcessing: {mission_name}")
    print("Preparing orthophoto for publishing.")

    # Is the Orthophoto from ODM or Pix4D?
    odm_ortho_path = os.path.join(
        mission_fold, "orthophoto", "odm_orthophoto.original.tif"
    )
    if os.path.isfile(odm_ortho_path):
        ortho_path = odm_ortho_path
    else:
        ortho_path = os.path.join(
            mission_fold, "orthophoto", "pix4d_orthophoto.original.tif"
        )

    # Standardise and save locally
    layer_name = sb.ortho.get_layer_name(mission_fold)
    temp_path = os.path.join(temp_dir, layer_name + ".tif")
    sb.geo.standardise_orthophoto(
        ortho_path,
        temp_path,
        red_band=1,
        green_band=2,
        blue_band=3,
        nodata=255,
    )

    # Copy to MinIO and delete local version
    stan_path = os.path.join(mission_fold, "orthophoto", layer_name + ".tif")
    sb.storage.copy_file(temp_path, stan_path, minio_client, overwrite=False)
    os.remove(temp_path)

    print("Uploading to GeoServer.")

    sb.geo.upload_raster_to_geoserver(
        stan_path,
        SETTINGS.GEOSERVER_USER,
        SETTINGS.GEOSERVER_PASSWORD,
        workspace="geonode",
    )

    print("Publishing to GeoNode.")

    sb.geo.publish_to_geonode(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        workspace="geonode",
    )

    print("Updating metadata.")
    date = sb.ortho.parse_mission_data(mission_fold, parse_date=True)[2]
    abstract = sb.geo.get_html_abstract(str(mission_fold))
    metadata = {
        "abstract": abstract,
        "date": date.isoformat(),
        "date_type": "creation",
        "attribution": "SeaBee",
    }
    sb.geo.update_geonode_metadata(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        metadata,
    )

NameError: name 'publish_list' is not defined

## 5. Failed tasks

In [12]:
print("The following missions have failed to process on NodeODM:")
nodeodm_tasks = sb.ortho.get_nodeodm_tasks(node)
for task in nodeodm_tasks:
    if str(task.info().status) == "TaskStatus.FAILED":
        print(task.info().name)

The following missions have failed to process on NodeODM:
/home/notebook/shared-seabee-ns9879k/seabirds/2023/oslofjorden_lyseren-zoom_202306271117
